In [1]:
import pickle

def load_embeddings_and_metadata(filename="data/pickle/embeddings_metadata_v2.pkl"):
    # Laden der Embeddings und Metadaten aus der Pickle-Datei
    with open(filename, "rb") as file:
        data = pickle.load(file)
    
    embeddings = data['embeddings']
    metadatas = data['metadatas']
    
    print(f"Die Embeddings und Metadaten wurden erfolgreich aus '{filename}' geladen.")
    return embeddings, metadatas

# Beispielnutzung:
embeddings, metadatas = load_embeddings_and_metadata(filename="data/pickle/embeddings_metadata_v2.pkl")

Die Embeddings und Metadaten wurden erfolgreich aus 'data/pickle/embeddings_metadata_v2.pkl' geladen.


In [2]:
def display_embeddings_and_metadata(embeddings, metadatas, num_examples=5):
    # Anzahl der Einträge, die Du anzeigen möchtest, begrenzen
    num_examples = min(num_examples, len(embeddings))  # Setzt num_examples auf 5 oder weniger, wenn nicht genug Embeddings vorhanden sind
    
    # Ausgabe der Embeddings und ihrer zugehörigen Metadaten
    for i in range(num_examples):
        print(f"Chunk {i}:\n")
        print(f"Embedding: {embeddings[i][:10]}...")  # Zeigt nur die ersten 10 Werte des Embeddings
        print(f"Metadaten: {metadatas[i]}")
        print("\n" + "="*50 + "\n")

# Beispielnutzung:
display_embeddings_and_metadata(embeddings, metadatas, num_examples=5)

Chunk 0:

Embedding: [-0.03529396 -0.2681408  -0.20776549  0.32234764  0.01191646 -0.3556172
  0.29910365  0.24749127  0.11207592  0.22963986]...
Metadaten: {'text': 'blaue altglascontainer saubere flaschen deckelgläser farben weiß blaues buntes glas grünglas geben aschen konservengläser scheiben restmüll trinkgläser porzellan keramik glühbirnen restmüll energiesparlampen wertstoffhöfe verschmutzte gläser restmüll standort öffentlichen sperrmüll plätzen frankfurter entsorgungs abfall trennen service gmbh fesservicecenter ecke töngesgasseschärfengässchen uhr uhr rgungsfachbetrie rrt trennen haus gehören abfallbehälter tonnen inhalt wertvoller rohstoff neuen produkten weiterverarbeitet graue restmülltonne gelbe tonne verpackungen grüne tonne papier pappe braune tonne bioabfälle graue tonne restlichen abfall restmüll gibt folgende entsorgungsangebote wertstoffe sichern altglascontainer flaschen deckelgläser nden öffentlichen plätzen altkleider container altkleider schuhe nden stadtteil öf

In [5]:
import spacy

# Lade das deutsche Sprachmodell von spaCy
nlp = spacy.load("de_core_news_sm")

def extract_keywords_from_query(query, custom_stopwords=None):
    """
    Extrahiert Keywords aus einer Query durch Tokenisierung und Entfernung von Stoppwörtern.
    """
    if custom_stopwords is None:
        custom_stopwords = set(nlp.Defaults.stop_words)
    
    # Tokenisiere die Query
    doc = nlp(query.lower())
    
    # Extrahiere Keywords: Wörter, die keine Stoppwörter, Zahlen oder Satzzeichen sind
    keywords = [token.text for token in doc if token.text not in custom_stopwords and not token.is_punct and not token.like_num]
    
    return keywords

# Beispielnutzung:
query2 = "Wie entsorge ich Altglas in Frankfurt?"
keywords2 = extract_keywords_from_query(query2)

# test chroma

#hybrid_results_v21 = hybrid_search_v2(chroma_client, query2, keywords2, embedding_model)

#print(f"Query: {query2}")
#print(f"Extrahierte Keywords: {keywords2}")
# Ausgabe der hybriden Suchergebnisse
#for i, result in enumerate(hybrid_results_v21):
#    print(f"Ergebnis {i+1}:\n")
 #   print(f"ID: {result['id']}")
  #  print(f"Text: {result['text']}")
   # print(f"Score: {result['score']}")
    #print("\n" + "="*50 + "\n")

In [4]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def create_chroma_database_v2(embeddings, metadatas, collection_name='document_embeddings_v3', persist_directory='chroma_db_v3'):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Erstelle eine neue Collection in der Datenbank oder lade eine bestehende
    collection = client.get_or_create_collection(name=collection_name)
    
    # Konvertiere Embeddings von numpy array in Liste von Listen
    embeddings_as_list = [embedding.tolist() for embedding in embeddings]
    
    # Füge die Embeddings und Metadaten zur Collection hinzu
    for i, (embedding, metadata) in enumerate(zip(embeddings_as_list, metadatas)):
        # Metadaten umwandeln, sodass die Liste der Keywords als String gespeichert wird
        metadata['keywords'] = ','.join(metadata['keywords'])
        
        collection.add(
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[f"doc_{i}"]
        )
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt und {len(embeddings)} Embeddings hinzugefügt.")

# Beispielnutzung:
create_chroma_database_v2(embeddings, metadatas)

Chroma-Datenbank 'document_embeddings_v3' erfolgreich erstellt und 10 Embeddings hinzugefügt.


In [3]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def delete_chroma_collection(collection_name='document_embeddings_v3', persist_directory='chroma_db_v3'):
    # Öffne die Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Überprüfe, ob die Collection existiert, und lösche sie, wenn ja
    existing_collections = [col.name for col in client.list_collections()]
    if collection_name in existing_collections:
        client.delete_collection(name=collection_name)
        print(f"Collection '{collection_name}' erfolgreich gelöscht.")
    else:
        print(f"Collection '{collection_name}' nicht gefunden.")
        
delete_chroma_collection(collection_name='document_embeddings_v3')

Collection 'document_embeddings_v3' nicht gefunden.


In [7]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# Lade das vortrainierte Modell
model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

# Initialisiere den Chroma-Client
chroma_client = chromadb.PersistentClient(
    path="chroma_db_v3",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)


# Überprüfe, ob die Collection existiert
def get_existing_collection(chroma_client, collection_name="document_embeddings_v3"):
    try:
        collection = chroma_client.get_collection(name=collection_name)
        print(f"Collection {collection_name} erfolgreich geladen.")
        return collection
    except ValueError:
        print(f"Collection {collection_name} existiert nicht.")
        return None

# Zugriff auf die bestehende Collection
collection = get_existing_collection(chroma_client, "document_embeddings_v3")

if collection is None:
    print("Die angegebene Collection existiert nicht. Bitte überprüfe den Collection-Namen.")
else:
    # Erweiterte Version der hybriden Suche mit normalisierter Distanzberechnung, Version 5
    def hybrid_search_v5(chroma_client, query, keywords, embedding_model, collection_name="document_embeddings_v3", top_k=10):
        # 1. Semantische Suche
        query_embedding = embedding_model.encode(query).tolist()
        collection = chroma_client.get_collection(name=collection_name)
        semantic_results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        # 2. Distanznormalisierung
        distances = semantic_results['distances'][0]
        min_distance = min(distances)
        max_distance = max(distances)
        normalized_distances = [(d - min_distance) / (max_distance - min_distance) for d in distances]
        
        # 3. Klassische Textsuche basierend auf Keywords
        text_search_results = []
        for i in range(len(semantic_results['ids'][0])):
            metadata = semantic_results['metadatas'][0][i]
            keyword_count = sum(keyword.lower() in metadata['text'].lower() for keyword in keywords)
            if keyword_count > 0:
                text_search_results.append({
                    'id': semantic_results['ids'][0][i],
                    'text': metadata['text'],
                    'keywords': keywords,
                    'score': 1.0 + 0.5 * keyword_count  # Gewichtung für Keywords
                })
        
        # 4. Kombination der Ergebnisse
        combined_results = []
        seen_ids = set()
        for i in range(len(semantic_results['ids'][0])):
            distance = normalized_distances[i]
            combined_score = 1.0 - distance  # Gewichtung für semantische Suche
            
            if semantic_results['ids'][0][i] not in seen_ids:
                seen_ids.add(semantic_results['ids'][0][i])
                combined_results.append({
                    'id': semantic_results['ids'][0][i],
                    'text': semantic_results['metadatas'][0][i]['text'] if isinstance(semantic_results['metadatas'][0][i], dict) else "",
                    'keywords': keywords,
                    'score': combined_score
                })
        
        for result in text_search_results:
            if result['id'] not in seen_ids:
                combined_results.append(result)
        
        # 5. Sortieren der kombinierten Ergebnisse nach Score
        combined_results.sort(key=lambda x: x['score'], reverse=True)
        
        # 6. Ausgabe der Top-k Ergebnisse
        return combined_results[:top_k]

    # Beispielnutzung:
    query5 = "Was kommt in die gelbe Tonne?"
    # query5 = "Wie entsorge ich Altglas in Frankfurt?"

    keywords5 = extract_keywords_from_query(query5)

    # Durchführung der hybriden Suche mit Keywords
    hybrid_results_v5 = hybrid_search_v5(chroma_client, query5, keywords5, embedding_model)

    print(f"Query: {query5}")
    print(f"Extrahierte Keywords: {keywords5}")
    # Ausgabe der hybriden Suchergebnisse
    for i, result in enumerate(hybrid_results_v5):
        print(f"Ergebnis {i+1}:\n")
        print(f"ID: {result['id']}")
        print(f"Text: {result['text']}")
        print(f"Keywords: {result.get('keywords', [])}")
        print(f"Score: {result['score']}")
        print("\n" + "="*50 + "\n")

Collection document_embeddings_v3 erfolgreich geladen.
Query: Was kommt in die gelbe Tonne?
Extrahierte Keywords: ['gelbe', 'tonne']
Ergebnis 1:

ID: doc_4
Text: beraten sack glascontainer fensterscheiben porzellan keramik glühbirnen uhr entsorgungsbetriebe uhr
Keywords: ['gelbe', 'tonne']
Score: 1.0


Ergebnis 2:

ID: doc_1
Text: nden jahr haushalte verteilt sperrmüllabholung holt fes termin wohnsiedlungen abgeholt termine nden schwarzen brett sperrmüll wertstoffe entsorgen hnff tie abfalltonnen haus dauer ausreichen hausbesitzer weiteregrößere bestellen internet spezialbehälter besondere abfälle baustellenabfälle erfahren servicetelefonnummer kosten sperrmüll abfallgebühr enthalten löse standorte fesmagazin erhalten wochen haus näheres gelbe verpackungstonne verpackungen symbol grünen papier pappe glas bestehen verpackungen gespült verpackungen kunststoff folien becher verbundverpackungen materialmix papier getränkekartons milchtüten verpackungen metall konserven getränkedosen geschä